In [3]:
import keras
import numpy as np
from PIL import Image
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
# from misc import load_data1, load_data2, load_data3, load_data4, load_data5, load_data6, load_data7, load_data8
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D

In [4]:
# data = load_data1()

K.set_learning_phase(1)

sess = K.get_session()
X = K.tf.placeholder(shape=(None, None, None, 3), dtype=K.tf.float32)
Y = K.tf.placeholder(shape=(None, 5), dtype=K.tf.float32)

In [5]:
vgg = VGGFace(include_top=False, input_tensor = X)
for layer in vgg.layers:
    layer.trainable = False

In [6]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, None, None, 64)    1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, None, None, 64)    36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, None, None, 64)    0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, None, None, 128)   73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, None, None, 128)   147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, None, None, 128)   0         
__________

In [7]:
p11 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=12).output))
p12 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=12).output)
p21 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=14).output))
p22 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=14).output)
p31 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(vgg.get_layer(index=16).output))
p32 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(vgg.get_layer(index=16).output)
p41 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(vgg.get_layer(index=18).output))
p42 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(vgg.get_layer(index=18).output)

In [27]:
p11

<tf.Tensor 'Sigmoid:0' shape=(?, ?, ?, 1) dtype=float32>

In [30]:
def filter_pred(scores = [p11, p21, p31, p41], bbox = [p12, p22, p32, p42]):
    
    p12, p22, p32, p42 = bbox
    
    p11_mask = K.greater_equal(scores[0], 0.5)
    p21_mask = K.greater_equal(scores[1], 0.5)
    p31_mask = K.greater_equal(scores[2], 0.5)
    p41_mask = K.greater_equal(scores[3], 0.5)
    
    p12 = K.tf.gather_nd(K.reshape(p12, (-1, 4)), indices= K.cast(p11_mask, dtype=K.tf.int32))
    p22 = K.tf.gather_nd(K.reshape(p22, (-1, 4)), indices= K.cast(p21_mask, dtype=K.tf.int32))
    p32 = K.tf.gather_nd(K.reshape(p32, (-1, 4)), indices= K.cast(p31_mask, dtype=K.tf.int32))
    p42 = K.tf.gather_nd(K.reshape(p42, (-1, 4)), indices= K.cast(p41_mask, dtype=K.tf.int32))
    
    return K.reshape(K.concatenate(tensors=[p12, p22, p32, p42], axis = 0), (-1, 4))

In [33]:
boxes = filter_pred()

In [9]:
def NMS(boxes):
    Z1 = K.tf.image.non_max_suppression(boxes=K.reshape(pred1[:, 1:], ((-1, 4))),
                                        scores=K.reshape(pred1[:, 0], ((-1,))), max_output_size=10,
                                        iou_threshold=0.55)
    Z2 = K.tf.image.non_max_suppression(boxes=K.reshape(pred2[:, 1:], ((-1, 4))),
                                        scores=K.reshape(pred2[:, 0], ((-1,))), max_output_size=10,
                                        iou_threshold=0.55)
    Z3 = K.tf.image.non_max_suppression(boxes=K.reshape(pred3[:, 1:], ((-1, 4))),
                                        scores=K.reshape(pred3[:, 0], ((-1,))), max_output_size=10,
                                        iou_threshold=0.55)
    Z4 = K.tf.image.non_max_suppression(boxes=K.reshape(pred4[:, 1:], ((-1, 4))),
                                        scores=K.reshape(pred4[:, 0], ((-1,))), max_output_size=10,
                                        iou_threshold=0.55)

    A = K.concatenate(tensors=[K.tf.dynamic_partition(pred1, Z1, 10)[1], K.tf.dynamic_partition(pred2, Z2, 10)[1], K.tf.dynamic_partition(pred3, Z3, 10)[1], K.tf.dynamic_partition(pred4, Z4, 10)[1]],
                      axis=0)

    Z5 = K.tf.image.non_max_suppression(boxes=K.reshape(A[:, 1:], (-1, 4)), scores=K.reshape(A[:, 0], (-1,)),
                                        max_output_size=10, iou_threshold=0.6)
    return K.gather(A, Z5)


In [15]:
def sigmoid(X):
    return 1/(1 + K.exp(-X))

def relu(X):
    return K.maximum(X, 0)

def loss(Y, y_, a=0.6):
    if K.tf.size(Y) != K.tf.size(y_):
        y_ = y_[:2, :]
    softMax = K.mean(-(Y[:, 0] * K.log(sigmoid(y_[:, 0])) + ((1 - Y[:, 0]) * K.log(sigmoid(1 - y_[:, 0])))))
    mse = K.mean(keras.losses.mean_squared_error(Y[:, 1:], relu(y_[:, 1:])))
    return softMax + a*mse 

In [ ]:
y_ = NMS(p1, p2, p3, p4)
sess.run(K.tf.global_variables_initializer())
h = sess.run(fetches=y_, feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis=0)})

# Loss = loss(Y, y_)

In [17]:
opt = K.tf.train.AdagradOptimizer(learning_rate=0.0001)
train_ = opt.minimize(Loss)
sess.run(K.tf.global_variables_initializer())

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [10]:
def train(epochs = 10):
    for e in range(epochs):
        cost = 0
#         for i, d in enumerate(data):
        _, L = sess.run([train_, Loss], feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis=0), Y: np.ones((2, 5)).reshape((-1, 5))})
        cost += L
        if e % 100 == 0:
            print("epoch {}, iter {} , loss {}".format(e, i , cost/len(data)))

In [ ]:
train(epochs = 12)

In [23]:
x = np.array([[1, 2, 3], [2, 3, 4], [3, 4, 5]])

In [28]:
s = np.array([0.43, 0.67, 0.5])

In [29]:
sm = np.greater_equal(s, 0.5)

In [30]:
x[sm]

array([[2, 3, 4],
       [3, 4, 5]])